In [ ]:
!git clone https://github.com/semantic-systems/t5-information-retrieval.git

In [ ]:
# Switch path to working dir
%cd t5-information-retrieval/question_generation-master/

# Install deps


In [ ]:
!pip install -r requirements.txt
!python -m nltk.downloader punkt

# Train data


In [ ]:
# Prepare the data for training
!python prepare_data.py --task qa --model_type t5 --dataset_path data/enron_train --qg_format highlight_qg_format --max_source_length 512 --max_target_length 32 --train_file_name enron_train_data_qg_hl_t5.pt --valid_file_name enron_valid_data_qg_hl_t5.pt

In [ ]:
# Start training
!python run_qg.py --model_name_or_path t5-small --model_type t5 \
    --tokenizer_name_or_path t5_qg_tokenizer \
    --output_dir t5-small-qg-hl \
    --train_file_path data/enron_train_data_qg_hl_t5.pt \
    --valid_file_path data/enron_valid_data_qg_hl_t5.pt \
    --per_device_train_batch_size 32 \
    --per_device_eval_batch_size 32 \
    --gradient_accumulation_steps 8 \
    --learning_rate 1e-4 \
    --num_train_epochs 5 \
    --seed 777 \
    --do_train \
    --do_eval \
    --evaluate_during_training \
    --logging_steps 100

# Use trained model

In [ ]:
import os

In [ ]:
from pipelines import pipeline
# Base
base_nlp = pipeline("multitask-qa-qg")
# Trained
trained_nlp = pipeline("multitask-qa-qg", model="./t5-small-qg-hl")

Load all the emails

In [ ]:
emails = []

# Filenames for emails to load from data/enron
emails_to_load = ["108_", "10_campbell", "153_", "83_", "10_", "12_"]
# If False, load all emails, otherwise load specified in emails_to_load
load_specified = False

file_path = 'data/enron'

if load_specified:
  for email in emails_to_load:
    with open(file_path + email) as f:
      contents = f.read()
      emails.append(contents)
else:
  for filename in os.listdir(file_path):
   # only files
   if not os.path.isfile(os.path.join(file_path, filename)):
     continue
   with open('data/enron/' + filename, 'r') as f:
      contents = f.read()
      emails.append(contents)

print("loaded all emails")

Finally use the model to work on the emails

In [ ]:
def get_model_answer(model, question, context):
  return model({
      'question': question,
      'context': context
  })

In [ ]:
def answer_question(question):
  for email in emails:
    base_answer = base_nlp({
      "question": question,
      "context": email
      })
    trained_answer = trained_nlp({
      "question": question,
      "context": email
      })
    print("Base Answer:", base_answer)
    print("Trained Answer:", trained_answer)

In [ ]:
answer_question("What is the subject of the document?")

In [ ]:
answer_question("Who sent the email?")

In [ ]:
answer_question("What events are described in the email?")

Try to display how well our model is doing compared to the annotated data.

In [ ]:
import json
valid_data_json = None
with open('data/enron_train/valid_output.json', 'r') as f:
      content = f.read()
      valid_data_json = json.loads(content)

Define some functions that should help us to evluate the answers given by the model

In [ ]:
import collections
import re
import string

# Use f1 score to compute the accuracy of an answer
# Copied from https://rajpurkar.github.io/SQuAD-explorer/ -> https://worksheets.codalab.org/rest/bundles/0x6b567e1cf2e041ec80d7098f031c5c9e/contents/blob/

def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()


def compute_f1(a_gold, a_pred):
  gold_toks = get_tokens(a_gold)
  pred_toks = get_tokens(a_pred)
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

In [ ]:
for document in valid_data_json['data']:
  for q_paragraph in document['paragraphs']:
    print('Document:')
    for question in q_paragraph['qas']:
      print('Question:', question['question'])
      print('Expected answers:', [ answer['text'] for answer in question['answers'] ])
      # [0] since the model (currently only gives one answer)
      model_answer = get_model_answer(trained_nlp, question['question'], q_paragraph['context'])[0]
      print('Model answer:', model_answer)
      print('f1 score for each annotated answer:', [compute_f1(model_answer, answer['text']) for answer in question['answers']])
      print('\n')